In [36]:
import requests
import re
from bs4 import BeautifulSoup as bs

In [37]:
def get_all_links_on_page(url):
    links = []
    response = requests.get(url)
    if response.status_code == 200:
        #extract links
        soup = bs(response.content, 'html.parser')
        content = soup.find("div", {"id":"mw-content-text"})
        for link in content.find_all("a"):
            links.append(link)
    return links

In [134]:
lists_of_people_links = get_all_links_on_page('https://en.wikipedia.org/wiki/Category:Lists_of_lists_of_people')
print(lists_of_people_links[0:2])

[<a href="/wiki/Category:Lists_of_entertainers_lists" title="Category:Lists of entertainers lists">Lists of entertainers lists</a>, <a href="/wiki/Category:Lists_of_nobility_lists" title="Category:Lists of nobility lists">Lists of nobility lists</a>]


We are interested in 4 kinds of links

* Lists of lists
* List of lists
* Lists of people
* person

In [130]:
def classify_link_type(wiki_link):
    regex = r"((?P<list_of_type>list[ _](?!lists).+)|(?P<lists_of_lists>lists[ _].+(?=lists).+)|(?P<lists_of_type>lists[ _](?!lists).+)|(?P<notable>notable.+(?!lists)))"
    try:
        results =  re.search(regex, wiki_link, re.IGNORECASE| re.MULTILINE).groupdict()
        match = [(k,v) for k,v in results.items() if v is not None].pop()
        return match
    except Exception as ee:
        return 'unknown'

In [131]:
classify_link_type('https://en.wikipedia.org/wiki/Lists_of_Armenians')

('lists_of_type', 'Lists_of_Armenians')

## The long path until PERSON page

### Start
https://en.wikipedia.org/wiki/Category:Lists_of_lists_of_people
### Lists of lists
https://en.wikipedia.org/wiki/Category:Lists_of_politicians_lists
### Lists of type
https://en.wikipedia.org/wiki/Lists_of_German_politicians
### List of type
https://en.wikipedia.org/wiki/List_of_German_National_People%27s_Party_politicians
### Person
https://en.wikipedia.org/wiki/Ernst_von_Dobsch%C3%BCtz

In [132]:
classify_link_type('https://en.wikipedia.org/wiki/Ernst_von_Dobsch%C3%BCtz')

'unknown'

In [136]:
for i in lists_of_people_links:
    x = classify_link_type(i['href'])
    if x == 'unknown':
        print(i)

<a href="/wiki/Former_FBI_Ten_Most_Wanted_Fugitives" title="Former FBI Ten Most Wanted Fugitives">Former FBI Ten Most Wanted Fugitives</a>
